In [102]:
import pandas as pd
import numpy as np
import os
import gc
import time
import re
import ast
import country_converter as coco
import time
import requests
import json
from urllib.parse import quote_plus
from collections import Counter
from functools import lru_cache

# Manage API Keys
from dotenv import load_dotenv
load_dotenv()

import logging
coco_logger = logging.getLogger('country_converter.country_converter')
coco_logger.setLevel(logging.ERROR)

In [2]:
DF_input = input().strip()

  C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\


In [3]:
entities_database_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\resources\allCountries_clean.csv


In [52]:
institutions_database_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\resources\ScimagoIR 2025 - Overall Rank.csv


In [112]:
df=pd.read_csv(DF_input+"parsedX_100000.csv")

In [5]:
entities_database = pd.read_csv(entities_database_input)

In [54]:
institutions_database = pd.read_csv(institutions_database_input, sep=";")

In [55]:
institutions_database

,Global Rank,Institution,Country,Sector,Best Country Quartile
0,1,Chinese Academy of Sciences *,CHN,Government,1
1,2,Ministry of Education of the People's Republic...,CHN,Government,1
2,3,Centre National de la Recherche Scientifique *,FRA,Government,1
3,4,Harvard University *,USA,Universities,1
4,5,University of Chinese Academy of Sciences *,CHN,Universities,1
...,...,...,...,...,...
9751,9752,Iranian Research Institute of Plant Protection,IRN,Government,4
9752,9753,Al-Turath University College,IRQ,Universities,4
9753,9754,Ecole Normale Superieure de Kouba Bachir el Ib...,DZA,Universities,4
9754,9755,Semashko National Research Institute of Public...,RUS,Health,4


In [57]:
# Remove " *" from institutions when present
institutions_database["Institution"] = institutions_database["Institution"].apply(lambda x: x[:-2] if x[-2:] == " *" else x)

In [6]:
entities_database.head()

,name,asciiname,alternatenames,country code
0,Xixerella,Xixerella,NaN,AD
1,Vila,Vila,"Casas Vila,Vila",AD
2,Tossalet i Vinyals1,Tossalet i Vinyals1,"Tossalet i Vin'jal's,Tossalet i Vinyals,Тоссал...",AD
3,Sornàs,Sornas,"Sornas,Sornàs",AD
4,Soldeu,Soldeu,"Sol'deu,Soldeu,surudeu,swldw,Сольдеу,סולדאו,سو...",AD


In [105]:
institutions_database.head()

,Global Rank,Institution,Country,Sector,Best Country Quartile
0,1,Chinese Academy of Sciences,CHN,Government,1
1,2,Ministry of Education of the People's Republic...,CHN,Government,1
2,3,Centre National de la Recherche Scientifique,FRA,Government,1
3,4,Harvard University,USA,Universities,1
4,5,University of Chinese Academy of Sciences,CHN,Universities,1


In [115]:
df.loc[df["PMID"]==1279132]["Authors"].values

array(["[{'Name': 'G Karp', 'Affiliation': ['Department of Medicine, UMDNJ-Robert Wood Johnson Medical School.']}, {'Name': 'K Nahum', 'Affiliation': []}]"],
      dtype=object)

In [ ]:
# Instance conerter
cc = coco.CountryConverter()

In [108]:
# Define a dictionary for US states. 
# Many article's affiliation have the state name/ code and not "United States"
us_states = [
    "Alabama", "AL", "Al",
    "Alaska", "AK", "Ak",
    "Arizona", "AZ", "Az",
    "Arkansas", "AR", "Ar",
    "California", "CA", "Ca",
    "Colorado", "CO", "Co",
    "Connecticut", "CT", "Ct",
    "Delaware", "DE", "De",
    "Florida", "FL", "Fl",
    "Georgia", "GA", "Ga",
    "Hawaii", "HI", "Hi",
    "Idaho", "ID", "Id",
    "Illinois", "IL", "Il",
    "Indiana", "IN", "In",
    "Iowa", "IA", "Ia",
    "Kansas", "KS", "Ks",
    "Kentucky", "KY", "Ky",
    "Louisiana", "LA", "La",
    "Maine", "ME", "Me",
    "Maryland", "MD", "Md",
    "Massachusetts", "MA", "Ma",
    "Michigan", "MI", "Mi",
    "Minnesota", "MN", "Mn",
    "Mississippi", "MS", "Ms",
    "Missouri", "MO", "Mo",
    "Montana", "MT", "Mt",
    "Nebraska", "NE", "Ne",
    "Nevada", "NV", "Nv",
    "New Hampshire", "NH", "Nh",
    "New Jersey", "NJ", "Nj",
    "New Mexico", "NM", "Nm",
    "New York", "NY", "Ny",
    "North Carolina", "NC", "Nc",
    "North Dakota", "ND", "Nd",
    "Ohio", "OH", "Oh",
    "Oklahoma", "OK", "Ok",
    "Oregon", "OR", "Or",
    "Pennsylvania", "PA", "Pa",
    "Rhode Island", "RI", "Ri",
    "South Carolina", "SC", "Sc",
    "South Dakota", "SD", "Sd",
    "Tennessee", "TN", "Tn",
    "Texas", "TX", "Tx",
    "Utah", "UT", "Ut",
    "Vermont", "VT", "Vt",
    "Virginia", "VA", "Va",
    "Washington", "WA", "Wa",
    "West Virginia", "WV", "Wv",
    "Wisconsin", "WI", "Wi",
    "Wyoming", "WY", "Wy",
    "District of Columbia", "DC", "Dc"
]

In [124]:
def extract_country_with_states(cell_content):
    # Extracting some countries requires parsing institutions in the institutions_database
    # Get all the institutions to parse:
    all_institutions = list(institutions_database["Institution"].values)
    
    # Avoid articles for which the NER provided no entities (i.e. articles without affiliation
    if pd.isna(cell_content) or cell_content is None:
        return None

    # In case the cell content is a list, proceed with extracting country
    elif isinstance(ast.literal_eval(cell_content), list):
        parsed_content = ast.literal_eval(cell_content)
        countries = []
        for el in parsed_content:
            if isinstance(el, dict):
                text = el["text"]
                label = el["label"]

                # Use list of US states to solve locations containing them directly
                if text in us_states:
                    return "United States"

                # Process only LOC and GPE labels with country-converter
                if label in ["LOC", "GPE"]:
                    standard_names = cc.convert(names = text, to = 'name_short')
                    if standard_names != "not found":
                        countries.append(standard_names)
            else:
                pass

        # If there is only one candidate country, return it
        if len(countries) == 1:
            return countries[0]

        # If there are no candidate countries using only country converter, move to other strategies
        elif len(countries) == 0:
            # Merge all texts into one string, in case needed for passing to code below
            string_parts = [el["text"] for el in parsed_content if isinstance(el, dict) and "text" in el]
            string = ", ".join(string_parts) + ", " if string_parts else ""
            string = string[:-2] # Remove last " ,"

            # Try to find the country by institution name
            # Loop through entities in a reverse manner (larger entities are found later in affiliations)
            parsed_content_org = [org for org in parsed_content if org["label"] == "ORG"]
            for el in parsed_content[::-1]:
                entity_name = el["text"]

                # If there is a direct hit, return it
                if all_institutions.count(entity_name) == 1:
                    return cc.convert(institutions_database[institutions_database["Institution"]==entity_name]["Country"].values[0], to="name_short")

                # If there are no hits, try again with regex
                elif all_institutions.count(entity_name) == 0:
                    possible_institutions = []
                    for institution in all_institutions:
                        finds_institution = re.findall(institution, entity_name)
                        if len(finds_institution) == 1:
                            possible_institutions.append(institution)
                    if len(possible_institutions) == 1:
                        return cc.convert(institutions_database[institutions_database["Institution"]==possible_institutions[0]]["Country"].values[0], to="name_short")

            # Try to fint the country by regional entity name
            # Loop through entities in a reverse manner (larger entities are found later in affiliations) 
            for el in parsed_content[::-1]:
                entity_name = el["text"]

                # Filter entities in the database
                df_entities = entities_database[entities_database["name"]==entity_name]

                # If there is only one, return it
                if len(df_entities) == 1:
                    return cc.convert(df_entities["country code"].iloc[0], to="name_short")

                # If there are more, try to find the country associated to each of them in the whole affiliation, and return it
                elif len(df_entities) > 1:
                    possible_countries = df_entities["country code"].values
                    for country in possible_countries:
                        if type(country)==str:
                            country_short_name = cc.convert(country, to="name_short")
                            if country_short_name != "not found" and re.search(r'\b' + re.escape(country_short_name) + r'\b', string):
                                return country_short_name
                                
            # If there are still no countries found, use online location services via APIs 
            # First use LocationIq
            # Do it only on the whole affiliation string first 
            
                return None

        # If there is more than a candidate country, return the candidate appearing more times
        else:
            count_country = {}
            for el in countries:
                if el in list(count_country.keys()):
                    count_country[el] += 1
                else:
                    count_country[el] = 1

            top_country = list(count_country.keys())[list(count_country.values()).index(max(count_country.values()))]
            return top_country

    # If no country is found, return None
    else:
        return None

start = time.time()
df_2["Country"] = df_2["NER_lastAuthor"].apply(extract_country_with_states)

print(time.time()-start)

119.49887704849243


In [117]:
len(df_2[(df_2["Country"].isna())&(df_2["NER_lastAuthor"].notna())])

312

In [113]:
df_2 = df.copy(deep=True)

In [114]:
df_2 = df_2.iloc[0:1000]

In [118]:
df_2[(df_2["Country"].isna())&(df_2["NER_lastAuthor"].notna())].head()

,Type,PMID,DOI,Journal,Title,Abstract,Authors,MeshHeadings,Chemicals,PublicationTypes,PublicationDate,Language,Keywords,NER_lastAuthor,Country
2,Article,1279132,10.1007/BF00172767,Journal of neuro-oncology,Hyperventilation as the initial manifestation ...,We present the case of a 57 year old man who d...,"[{'Name': 'G Karp', 'Affiliation': ['Departmen...",Antineoplastic Combined Chemotherapy Protocols...,"['Vincristine', 'Doxorubicin', 'Cyclophosphami...","['Case Reports', 'Journal Article']",1992.0,eng,NaN,"[{'text': 'Department of Medicine', 'label': '...",None
6,Article,1279163,10.1111/j.1600-0714.1992.tb01030.x,Journal of oral pathology & medicine : officia...,Reactivity of monoclonal antibodies 17.13 and ...,We studied the reactivity of monoclonal antibo...,"[{'Name': 'C A Migliorati', 'Affiliation': ['D...","Antibodies, Monoclonal, Carcinoma, Squamous Ce...","['Antibodies, Monoclonal']",['Journal Article'],1992.0,eng,NaN,[{'text': 'Department of Oral Diagnostic Scien...,None
7,Article,1279165,NaN,Journal de radiologie,[Primary mediastinal choriocarcinoma. Apropos ...,We report three cases about the diagnostic ima...,"[{'Name': 'J M Girault', 'Affiliation': [""Serv...","Adult, Antineoplastic Combined Chemotherapy Pr...","['Bleomycin', 'Etoposide', 'Cisplatin']","['Case Reports', 'Journal Article']",1992.0,fre,NaN,"[{'text': 'Service de Radiologie', 'label': 'O...",None
8,Article,1279183,10.1093/jnci/84.21.1633,Journal of the National Cancer Institute,Role of the SIKVAV site of laminin in promotio...,Angiogenesis (vascularization) has a critical ...,"[{'Name': 'M C Kibbey', 'Affiliation': ['Labor...","Amino Acid Sequence, Animals, Binding Sites, C...","['Drug Combinations', 'Laminin', 'Oligopeptide...",['Journal Article'],1992.0,eng,NaN,[{'text': 'Laboratory of Developmental Biology...,None
14,Article,1279192,10.1007/BF02887790,Journal of Tongji Medical University = Tong ji...,Studies on monoclonal anti-isotypic and anti-i...,In this study the technique of labelling the c...,"[{'Name': 'X L Wang', 'Affiliation': ['Departm...","Antibodies, Anti-Idiotypic, Antibodies, Monocl...","['Antibodies, Anti-Idiotypic', 'Antibodies, Mo...","['Journal Article', ""Research Support, Non-U.S...",1992.0,eng,NaN,"[{'text': 'Department of Immunology', 'label':...",None


In [123]:
string_parts = [el["text"] for el in ast.literal_eval(df_2.iloc[2]["NER_lastAuthor"]) if isinstance(el, dict) and "text" in el]
string = ", ".join(string_parts) + ", " if string_parts else ""
string = string[:-2]
string

'Department of Medicine, UMDNJ-Robert Wood Johnson Medical School'